## __RANGE-BOUND STABILITY MODEL__

In [16]:
import pandas as pd
import numpy as np
import random
import json

from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
pd.options.plotting.backend = "plotly"

from src.utils import ModelParams, Day, short_sin, short_cos, long_sin, long_cos
from src.init_functions import initial_params

### __SET SCENARIO PARAMETERS__

In [17]:
# Simulate scenario with RBS market operations

netflow_type, historical_net_flows, price, target, supply, reserves, reserves_volatile, liq_usd = initial_params(
    netflow_type = 'random' # Determines the netflow types. Either 'historical', 'enforced', 'random', or 'cycles' (sin/cos waves)
    #,initial_date = '2021/12/18' # Determines the initial date to account for 'historical' netflows and initial params. (example: '2021/12/18')
    #,netflow_data = 'test'
    ,initial_supply = 30000000
    ,initial_reserves_usd = 200000000
    ,initial_reserves_volatile = 17000000
    ,initial_liq_usd = 21000000
    ,initial_price = 10.5
    ,initial_target = 10.5
)

params = ModelParams(seed = 69 # Seed number to control simulation randomness
    ,horizon = 365  # Simulation timespan
    ,short_cycle = 30  # Short market cycle duration (only relevant for netflow_type == cycles)
    ,cycle_reweights = 1  # Reweights per short market cycle (Deprecated)
    ,long_cycle = 730  # Long market cycle duration (only relevant for netflow_type == cycles)
    ,long_sin_offset = 2  # Demand function offset (only relevant for netflow_type == cycles)
    ,long_cos_offset = 0  # Supply function offset (only relevant for netflow_type == cycles)
    ,supply_amplitude = 0.8  # Supply function amplitude (only relevant for netflow_type == cycles)
    ,with_reinstate_window = 'Yes'
    ,with_dynamic_reward_rate = 'No'

    # Initial Parameters
    ,initial_supply = supply, initial_reserves_usd = reserves, initial_reserves_volatile = reserves_volatile, initial_liq_usd = liq_usd, initial_price = price, initial_target = target, target_price_function = 'price_moving_avg', netflow_type = netflow_type

    ,demand_factor = -0.008 # % of OHM supply expected to drive market demand
    ,supply_factor = 0.0081  # % of OHM supply expected to drive market sell preasure
    ,arb_factor = 0  # Initial arb factor
    ,release_capture = 0  # % of reweight taken immediately by the market (Deprecated)

    ,max_liq_ratio = 0.14375  # LiquidityUSD : reservesUSD ratio --> 1:1 = 0.5
    ,min_premium_target = 0  # Minimum premium for mint&sync --> to keep adding liquidity as supply grows 
    ,max_outflow_rate = 0.05  # Max % of reservesUSD that can be released on a single day
    ,reserve_change_speed = 1  # Directly related to the speed at which reserves are released/captured by the treasury. The higher the slower

    ,ask_factor = 0.095  # % of floating supply that the treasury can deploy when price is trading above the upper target
    ,bid_factor = 0.095  # % of the reserves that the treasury can deploy when price is trading below the lower target
    ,cushion_factor = 0.3075  # The percentage of a bid or ask to offer as a cushion
    ,target_ma = 30  # Length of the price target moving average (in days)
    ,lower_wall = 0.295  # Determines lower wall price target at x% below the target price
    ,upper_wall = 0.295  # Determines upper wall price target at x% above the target price
    ,lower_cushion = 0.1675  # Determines lower cushion price target at x% below the target price
    ,upper_cushion = 0.1675  # Determines upper cushion price target at x% above the target price
    ,reinstate_window = 7  # The window of time (in days) to reinstate a bid or ask
    ,min_counter_reinstate = 6  # Number of days within the reinstate window that conditions are true to reinstate a bid or ask
)

lags = {
    'price': (0, {1: params.initial_price}), 'target': (0, {1: params.initial_target}), 'avg': (0, {1: params.initial_target}), 'gohm price variation': (0, {1: 0})
}

arbs = {}
rbs_netflows = [0]

random.seed(params.seed)

if historical_net_flows is None:
    simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags, historical_net_flows=None)}
    for i in range (2, params.horizon):
        simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'], historical_net_flows=None)
        rbs_netflows.append(max(simulation[f'day{i}'].net_flow, 0.9*simulation[f'day{i}'].net_flow))
else:
    simulation = {'day1': Day(params=params, prev_arbs=arbs, prev_lags=lags, historical_net_flows=historical_net_flows[0])}
    for i in range (2, min(params.horizon, len(historical_net_flows) - 1)):
        simulation[f'day{i}'] = Day(params=params, prev_arbs=arbs, prev_lags=lags, prev_day=simulation[f'day{i-1}'], historical_net_flows=historical_net_flows[i-1])

In [18]:
# Simulate same scenario (same netflows) WITHOUT RBS market operations
params_without = ModelParams(seed = params.seed
    ,horizon = params.horizon
    ,short_cycle = params.short_cycle
    ,cycle_reweights = params.cycle_reweights
    ,long_cycle = params.long_cycle
    ,long_sin_offset = params.long_sin_offset
    ,long_cos_offset = params.long_cos_offset
    ,supply_amplitude = params.supply_amplitude
    ,with_reinstate_window = params.with_reinstate_window
    ,with_dynamic_reward_rate = params.with_dynamic_reward_rate
    
    # Initial Parameters
    ,initial_supply = params.initial_supply, initial_reserves_usd = params.initial_reserves_usd, initial_reserves_volatile = params.initial_reserves_volatile, initial_liq_usd = params.initial_liq_usd, initial_price = params.initial_price, initial_target = params.initial_target, target_price_function = params.target_price_function
    ,netflow_type = 'enforced'

    ,demand_factor = params.demand_factor
    ,supply_factor = params.supply_factor
    ,arb_factor = params.arb_factor
    ,release_capture = params.release_capture

    ,max_liq_ratio = params.max_liq_ratio
    ,min_premium_target = params.min_premium_target
    ,max_outflow_rate = params.max_outflow_rate  # Set to 0 to do not rebalance, set to params.max_outflow_rate to mimic RBS
    ,reserve_change_speed=params.reserve_change_speed

    ,ask_factor = 0
    ,bid_factor = 0
    ,cushion_factor = 0
    ,target_ma = 30
    ,lower_wall = 0
    ,upper_wall = 0
    ,lower_cushion = 0
    ,upper_cushion = 0
    ,reinstate_window = 0
    ,min_counter_reinstate = 0
)

lags_without = {
    'price': (0, {1: params_without.initial_price}), 'target': (0, {1: params_without.initial_target}), 'avg': (0, {1: params_without.initial_target}), 'gohm price variation': (0, {1: 0})
}

arbs_without = {}

simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without)}
theoretical_supply = [supply]

if historical_net_flows is None:
    simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without, historical_net_flows=rbs_netflows[0])}
    for i in range (2, params.horizon):
        simulation_without[f'day{i}'] = Day(params=params_without, prev_arbs=arbs_without, prev_lags=lags_without, prev_day=simulation_without[f'day{i-1}'], historical_net_flows=rbs_netflows[i-1])
        theoretical_supply += [theoretical_supply[-1] * 1.000198]
else:
    simulation_without = {'day1': Day(params=params_without, prev_arbs=arbs, prev_lags=lags_without, historical_net_flows=historical_net_flows[0])}
    for i in range (2, min(params.horizon, len(historical_net_flows) - 1)):
        simulation_without[f'day{i}'] = Day(params=params_without, prev_arbs=arbs_without, prev_lags=lags_without, prev_day=simulation_without[f'day{i-1}'], historical_net_flows=historical_net_flows[i-1])
        theoretical_supply += [theoretical_supply[-1] * 1.000198]

### __PLOT RESULTS__

In [19]:
# Protocol variables
df1 = pd.DataFrame(columns = ['Type', 'NetFlow', 'CumNetFlow', 'Price', '30dMA', 'LiqBacking', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'TheoreticalSupply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumMintedOHM', 'CumBurntOHM']) 
for day, data in simulation.items():
    cum  = cum + float(data.net_flow) if day != 'day1' else float(data.net_flow)
    df1.loc[day] = ['Mint&Sync + TreasuryRebalance + MarketOps', float(data.net_flow), cum, float(data.price), float(data.ma_target), float(data.lb_target), float(data.lower_target_cushion), float(data.upper_target_cushion), float(data.lower_target_wall), float(data.upper_target_wall), float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), theoretical_supply[int(day[3:])-1], float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_minted), float(data.cum_ohm_burnt)]


df2 = pd.DataFrame(columns = ['Type', 'NetFlow', 'Price', '30dMA', 'LiqBacking', 'LowerTargetCushion', 'UpperTargetCushion', 'LowerTargetWall', 'UpperTargetWall', 'LiqUSD', 'LiqOHM', 'poolK', 'Reserves', 'ReserveChange', 'ReservesIN', 'ReservesOUT', 'TradedOHM', 'Treasury', 'Supply', 'MCap', 'FloatingSupply', 'FloatingMCap', 'LiqRatio (Liq/Treasury)', 'LiqRatio (Liq/Reserves)', 'ReserveRatio', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium', 'CumMintedOHM', 'CumBurntOHM']) 
for day, data in simulation_without.items():
        df2.loc[day] = ['Mint&Sync + TreasuryRebalance (withoutMarketOps)', float(data.net_flow), float(data.price), float(data.ma_target), float(data.lb_target), np.nan, np.nan, np.nan, np.nan, float(data.liq_usd), float(data.liq_ohm), float(data.k), float(data.reserves), float(100*data.reserves/data.prev_reserves), float(data.reserves_in), float(data.reserves_out), float(data.ohm_traded), float(data.treasury), float(data.supply), float(data.mcap), float(data.floating_supply), float(data.floating_mcap), float(data.liq_ratio), float(data.liq_usd/data.reserves), float(data.reserves_ratio), float(data.liq_fmcap_ratio), float(data.fmcap_treasury_ratio), float(data.cum_ohm_minted), float(data.cum_ohm_burnt)]
df=pd.concat([df1, df2])

# Guidance variables
guidance_df = pd.DataFrame(columns = ['BidCapacity', 'AskCapacity', 'BidCapacityCushion', 'AskCapacityCushion', 'BidCapacityTargetCushion', 'AskCapacityTargetCushion', 'BidCapacityTarget', 'AskCapacityTarget', 'TradedOHM', 'ReservesOUT', 'AskCount', 'BidCount']) 
for day, data in simulation.items():
    guidance_df.loc[day] = [data.bid_capacity, data.ask_capacity * data.upper_target_wall, data.bid_capacity_cushion, data.ask_capacity_cushion * data.upper_target_cushion, data.bid_capacity_target_cushion, data.ask_capacity_target_cushion * data.upper_target_cushion, data.bid_capacity_target, data.ask_capacity_target * data.upper_target_wall, data.ohm_traded, data.reserves_out, data.control_ask, data.control_bid]

# Market dynamics variables
market_df = pd.DataFrame(columns = ['MarketDemand', 'MarketSupply', 'NetTotal'])
for day, data in simulation.items():
    market_df.loc[day] = [data.market_demand, data.market_supply, data.total_net]

plot_horizon = 365

In [20]:
# Plot bid/ask/price charts

comp1a = go.Figure()
comp1a.add_trace(go.Scatter(y=df1['Price'], x=list(df1.index), name='Price'))
comp1a.add_trace(go.Scatter(y=df1['30dMA'], x=list(df1.index), name='Target MA'))
comp1a.add_trace(go.Scatter(y=df1['LiqBacking'], x=list(df1.index), name='Liq Backing'))
comp1a.add_trace(go.Scatter(y=df1['UpperTargetWall'], x=list(df1.index), name='High Wall', line=dict(color='grey', dash='dot')))
comp1a.add_trace(go.Scatter(y=df1['LowerTargetWall'], x=list(df1.index), name='Low Wall', line=dict(color='grey', dash='dot')))
comp1a.add_trace(go.Scatter(y=df1['UpperTargetCushion'], x=list(df1.index), name='High Cushion', line=dict(color='silver', dash='dot')))
comp1a.add_trace(go.Scatter(y=df1['LowerTargetCushion'], x=list(df1.index), name='Low Cushion', line=dict(color='silver', dash='dot')))
comp1a.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
comp1a.show()

comp2 = guidance_df[['AskCapacity', 'AskCapacityTarget', 'AskCapacityCushion', 'AskCapacityTargetCushion']].plot()
comp2.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp2.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp2.show()

comp2b = guidance_df[['AskCount']].plot()
comp2b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp2b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    comp2b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#comp2b.show()

comp1b = comp1a
comp1b.show()

comp3 = guidance_df[['BidCapacity', 'BidCapacityTarget', 'BidCapacityCushion', 'BidCapacityTargetCushion']].plot()
comp3.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp3.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp3.show()


comp3b = guidance_df[['BidCount']].plot()
comp3b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp2b.add_vline(x=i, line_width=1, line_color="white", layer='below')
    comp2b.add_hline(y=params.min_counter_reinstate, line_width=0.5, line_dash="dash", line_color="grey")
#comp2b.show()

In [21]:
# Plot batch1

for col in df1.columns:
    if col in ('NetFlow', 'CumNetFlow', 'LiqRatio (Liq/Treasury)', 'LiqFloatingMCRatio', 'FloatingMCTreasuryPremium'):
        comp = df1.plot(x=list(df1.index), y=df1[col])
        comp.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            comp.add_vline(x=i, line_width=1, line_color="white", layer='below')
        if col == 'LiqRatio (Liq/Treasury)':
            comp.add_hline(y=params.max_liq_ratio, line_width=1, line_dash="dash", line_color="grey")
        if col == 'FloatingMCTreasuryPremium':
            comp.add_hline(y=params.min_premium_target, line_width=1, line_dash="dash", line_color="grey")
        comp.show()

In [22]:
# Plot batch2
for col in df1.columns:
    if col in ('Reserves', 'LiqUSD', 'LiqOHM', 'ReservesIN', 'ReservesOUT', 'Treasury'):
        comp = df1.plot(x=list(df1.index), y=df1[col])
        comp.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            comp.add_vline(x=i, line_width=1, line_color="white", layer='below')
        #comp.show()

In [23]:
# Plot batch3
comp3a = df1[['Supply', 'TheoreticalSupply']].plot()
comp3a.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp3a.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp3a.show()

for col in df1.columns:
    if col in ('FloatingSupply', 'CumMintedOHM', 'CumBurntOHM'):
        comp3b = df1.plot(x=list(df1.index), y=df1[col])
        comp3b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=False))
        for i in range (params.short_cycle, plot_horizon, params.short_cycle):
            comp3b.add_vline(x=i, line_width=1, line_color="white", layer='below')
        comp3b.show()

In [24]:
# Plot multivariable charts - Comparison vs without market operations

comp = df[['Treasury', 'LiqUSD', 'Reserves', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
comp.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
comp.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp.show()


comp1 = df[['MCap','FloatingMCap', 'LiqUSD', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
comp1.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
comp1.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp1.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp1.show()


comp2 = df[['Price','RealTarget', 'LowerTargetWall', 'UpperTargetWall', 'LowerTargetCushion', 'UpperTargetCushion', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
comp2.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
comp2.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp2.add_vline(x=i, line_width=1, line_color="white", layer='below')
comp2.show()


comp3 = df[['Supply','FloatingSupply', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
comp3.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
comp3.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp3.add_vline(x=i, line_width=1, line_color="white", layer='below')
#comp3.show()


comp3b = df[['CumBurntOHM', 'CumMintedOHM', 'Type']].plot(facet_col='Type', facet_col_wrap=2)
comp3b.layout.update(xaxis=dict(showgrid=False), yaxis=dict(showgrid=True), xaxis2=dict(showgrid=False), yaxis2=dict(showgrid=True))
comp3b.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
for i in range (params.short_cycle, plot_horizon, params.short_cycle):
    comp3b.add_vline(x=i, line_width=1, line_color="white", layer='below')
#comp3b.show()


KeyError: "['RealTarget'] not in index"

In [ ]:
# Market cycles (Only applicable if netflow_type == cycles --> sin and cos waves)
df2 = pd.DataFrame(columns = ['shortSin', 'shortCos', 'longSin', 'longCos']) 
for i in range (2, 10*params.horizon):
    df2.loc[f'day{i}'] = [short_sin(i, params.short_cycle), short_cos(i, params.short_cycle), long_sin(i, params.long_cycle, params.long_sin_offset), long_cos(i, params.long_cycle, params.long_cos_offset, params.supply_amplitude)]

#df2.plot(y=df2.columns)

In [ ]:
'''
vs = True

if vs is True:

    rbs = []
    for n in rbs_netflows:
        d = dict()
        d["netFlow"]=str(n)
        rbs.append(d)

    with open("./data/sim-vs-testnet/sim-results-test.json", "w") as write_file:
        write_file.write(json.dumps(rbs))

    f = open(f'./data/sim-vs-testnet/sim-results-test.json')
    data = json.load(f)
    df = pd.json_normalize(data)
    f.close()
    historical_net_flows = df['netFlow'].apply(lambda x: round(float(x),2)).tolist()
    historical_net_flows
'''